In [517]:
!pip install nltk gensim


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [518]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [519]:
df = pd.read_csv("../data/gpt_fake_recommendations.csv")

In [520]:
df.head()

,Company,Short Name,Date,Target Price,Recommendation,Commentary
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth in cost management, c..."
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL is benefiting from investment in research...
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,AAPL's performance in technology has been stea...
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties in shift in consumer ...
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges in expansion into internatio...


In [521]:
df.value_counts("Recommendation")

Recommendation
Buy     434
Sell    428
Hold    374
Name: count, dtype: int64

In [522]:
df.value_counts("Company")

Company
3M Company                           12
Norfolk Southern Corporation         12
Salesforce, Inc.                     12
S&P Global Inc.                      12
Raytheon Technologies Corporation    12
                                     ..
Exelon Corporation                   12
Eli Lilly and Company                12
Dow Inc.                             12
Deere & Company                      12
Zoetis Inc.                          12
Name: count, Length: 103, dtype: int64

In [523]:
df.groupby(["Short Name"]).mean("Target Price")

,Target Price
Short Name,
AAPL,249.782500
ABBV,227.696667
ADBE,251.938333
ALL,280.011667
AMGN,294.452500
...,...
VZ,259.306667
WFC,246.332500
WMT,251.254167


In [524]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Company         1236 non-null   object 
 1   Short Name      1236 non-null   object 
 2   Date            1236 non-null   object 
 3   Target Price    1236 non-null   float64
 4   Recommendation  1236 non-null   object 
 5   Commentary      1236 non-null   object 
dtypes: float64(1), object(5)
memory usage: 58.1+ KB


# Removing Stopwords

In [525]:
df["Commentary"] = df["Commentary"].apply(strip_multiple_whitespaces)

In [526]:
# Show list of stopwords
nltk_stopwords = set(stopwords.words('english'))
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some',

In [527]:
# Remove negations from the list
to_remove=['ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'don', "don't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [528]:
for word in to_remove:
    nltk_stopwords.remove(word)

In [529]:
print(sorted(nltk_stopwords))

['a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'd', 'did', 'do', 'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'she', "she's", 'should', "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'who

In [530]:
# Correct apply version
df["Commentary"] = df["Commentary"].apply(lambda comment: remove_stopwords(comment, stopwords=nltk_stopwords))

In [531]:
print(df)

                 Company Short Name        Date  Target Price Recommendation  \
0             Apple Inc.       AAPL  2023-01-15        121.35           Hold   
1             Apple Inc.       AAPL  2023-02-15        371.59            Buy   
2             Apple Inc.       AAPL  2023-03-15        202.52           Hold   
3             Apple Inc.       AAPL  2023-04-15        355.88           Hold   
4             Apple Inc.       AAPL  2023-05-15        292.45           Sell   
...                  ...        ...         ...           ...            ...   
1231  Exelon Corporation        EXC  2023-08-15        242.37           Sell   
1232  Exelon Corporation        EXC  2023-09-15        248.48            Buy   
1233  Exelon Corporation        EXC  2023-10-15        148.36           Hold   
1234  Exelon Corporation        EXC  2023-11-15        143.41            Buy   
1235  Exelon Corporation        EXC  2023-12-15        172.69            Buy   

                                       

# Add dates for 3 / 6 / 9 / 12 months

In [532]:
# transform data column to actual date
df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         1236 non-null   object        
 1   Short Name      1236 non-null   object        
 2   Date            1236 non-null   datetime64[ns]
 3   Target Price    1236 non-null   float64       
 4   Recommendation  1236 non-null   object        
 5   Commentary      1236 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 58.1+ KB


In [533]:
# Apply DateOffset for each row using 'apply' on the DataFrame
df["Date next Day"] = df["Date"].apply(lambda date: date + pd.DateOffset(days=1))
df["Date after 3 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=3))
df["Date after 6 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=6))
df["Date after 9 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=9))
df["Date after 12 Months"] = df["Date"].apply(lambda date: date + pd.DateOffset(months=12))


In [534]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Company               1236 non-null   object        
 1   Short Name            1236 non-null   object        
 2   Date                  1236 non-null   datetime64[ns]
 3   Target Price          1236 non-null   float64       
 4   Recommendation        1236 non-null   object        
 5   Commentary            1236 non-null   object        
 6   Date next Day         1236 non-null   datetime64[ns]
 7   Date after 3 Months   1236 non-null   datetime64[ns]
 8   Date after 6 Months   1236 non-null   datetime64[ns]
 9   Date after 9 Months   1236 non-null   datetime64[ns]
 10  Date after 12 Months  1236 non-null   datetime64[ns]
dtypes: datetime64[ns](6), float64(1), object(4)
memory usage: 106.3+ KB


# API Connection

In [535]:
"""import yfinance as yf
from datetime import timedelta
import pandas as pd
import numpy as np  # Import numpy to use np.abs


#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!

def get_stock_price_nearest(ticker, date):
    try:
        # Ensure the date is in Pandas Timestamp format
        target_date = pd.to_datetime(date)

        # Define a range around the target date to fetch data
        start_date = target_date - timedelta(days=5)
        end_date = target_date + timedelta(days=5)

        # Download data within this range
        stock_data = yf.download(ticker, start=start_date, end=end_date)

        if not stock_data.empty:
            # Check if data exists for the exact date
            if target_date in stock_data.index:
                return stock_data.loc[target_date]['Close']
            else:
                # Manually find the closest available trading day using numpy.abs()
                stock_data['diff'] = np.abs(stock_data.index - target_date)
                closest_date = stock_data['diff'].idxmin()  # Get the index of the closest date
                closest_price = stock_data.loc[closest_date]['Close']
                return closest_price
        else:
            return None
    
    except Exception as e:
        return f"An unexpected error occurred for {ticker}: {str(e)}"

#def get_max_stock_price_for_timeframe(short_name,time_frame):
    """

'import yfinance as yf\nfrom datetime import timedelta\nimport pandas as pd\nimport numpy as np  # Import numpy to use np.abs\n\n\n#this function might not be necessary since it makes more sense to look at the whole time frame to see if a target price was reached or not!\n\ndef get_stock_price_nearest(ticker, date):\n    try:\n        # Ensure the date is in Pandas Timestamp format\n        target_date = pd.to_datetime(date)\n\n        # Define a range around the target date to fetch data\n        start_date = target_date - timedelta(days=5)\n        end_date = target_date + timedelta(days=5)\n\n        # Download data within this range\n        stock_data = yf.download(ticker, start=start_date, end=end_date)\n\n        if not stock_data.empty:\n            # Check if data exists for the exact date\n            if target_date in stock_data.index:\n                return stock_data.loc[target_date][\'Close\']\n            else:\n                # Manually find the closest available trad

In [536]:
import pandas as pd

# Load and ensure 'Date' is in datetime format
df_performance_data = pd.read_csv("../data/performance_data.csv")

# Convert 'Date' column to datetime and set it as index (with timezone awareness)
df_performance_data['Date'] = pd.to_datetime(df_performance_data['Date'], utc=True)
df_performance_data = df_performance_data.set_index('Date')

# Function to get the maximum stock price within a date range or the nearest available date
def get_max_stock_price(ticker, start_date, end_date=None):
    try:
        # Ensure start_date and end_date are in the same timezone (UTC)
        start_date = pd.to_datetime(start_date, utc=True)
        if end_date is not None:
            end_date = pd.to_datetime(end_date, utc=True)
        else:
            end_date = start_date  # If no end_date, use start_date for a single day

        # Filter the data for the ticker and date range
        filtered_data = df_performance_data.loc[
            (df_performance_data.index >= start_date) & 
            (df_performance_data.index <= end_date), ticker]

        if not filtered_data.empty:
            # Return the maximum price within the filtered date range
            max_price = filtered_data.max()
            return float(max_price)
        else:
            # If no data available in the range, find the next available date using asof()
            next_available_data = df_performance_data[ticker].asof(start_date)
            if next_available_data is not None:
                return float(next_available_data)
            else:
                return float('nan')  # Return NaN for missing data
    
    except KeyError:
        return float('nan')  # Return NaN for missing data
    except Exception as e:
        return float('nan')  # Return NaN for missing data

# Example function call
print(get_max_stock_price(df["Short Name"][0], df["Date"][0], df["Date next Day"][0]))


133.50433349609375


In [537]:
print(get_max_stock_price(df["Short Name"][0], df["Date"][0], df["Date after 3 Months"][0]))

164.8730010986328


In [538]:
def calculate_prices(row):
    short_name = row['Short Name']
    row['Start Price'] = get_max_stock_price(short_name, row['Date'])
    row['One Day after'] = get_max_stock_price(short_name,row['Date next Day'])
    row['Price after 3 Months'] = get_max_stock_price(short_name, row['Date'],row['Date after 3 Months'])
    row['Price after 6 Months'] = get_max_stock_price(short_name,row['Date after 3 Months'], row['Date after 6 Months'])
    row['Price after 9 Months'] = get_max_stock_price(short_name, row['Date after 6 Months'], row['Date after 9 Months'])
    #row['Price after 12 Months'] = get_max_stock_price(short_name, row['Date after 12 Months'])
    return row
# Apply the function to each row
df = df.apply(calculate_prices, axis=1)

In [539]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One Day after,Price after 3 Months,Price after 6 Months,Price after 9 Months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,133.504333,133.504333,164.873001,192.722244,195.186279
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,2024-02-15,154.117630,152.510254,172.393860,195.186279,188.734222
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,2024-03-15,151.795898,154.633575,184.813461,195.186279,197.361084
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,2024-04-15,163.920502,163.920502,192.722244,195.186279,197.361084
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,2024-05-15,170.963104,170.963104,195.186279,188.734222,197.361084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,2024-08-15,38.032528,38.243870,40.194012,39.755924,38.337326
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,2024-09-15,40.011482,40.011482,40.011482,36.447979,38.337326
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,2024-10-15,38.378361,38.589710,39.755924,37.114410,38.337326
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,2024-11-15,38.466282,38.282047,39.755924,38.337326,38.376923


In [540]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Company               1236 non-null   object        
 1   Short Name            1236 non-null   object        
 2   Date                  1236 non-null   datetime64[ns]
 3   Target Price          1236 non-null   float64       
 4   Recommendation        1236 non-null   object        
 5   Commentary            1236 non-null   object        
 6   Date next Day         1236 non-null   datetime64[ns]
 7   Date after 3 Months   1236 non-null   datetime64[ns]
 8   Date after 6 Months   1236 non-null   datetime64[ns]
 9   Date after 9 Months   1236 non-null   datetime64[ns]
 10  Date after 12 Months  1236 non-null   datetime64[ns]
 11  Start Price           936 non-null    float64       
 12  One Day after         936 non-null    float64       
 13  Price after 3 Mont

# Calculate performance of Target Prices

In [541]:
df["tp_reached_after_3_months"] = df["Target Price"] <= df["Price after 3 Months"]
df["tp_reached_after_6_months"] = df["Target Price"] <= df["Price after 6 Months"]
df["tp_reached_after_9_months"] = df["Target Price"] <= df["Price after 9 Months"]

In [545]:
df

,Company,Short Name,Date,Target Price,Recommendation,Commentary,Date next Day,Date after 3 Months,Date after 6 Months,Date after 9 Months,Date after 12 Months,Start Price,One Day after,Price after 3 Months,Price after 6 Months,Price after 9 Months,tp_reached_after_3_months,tp_reached_after_6_months,tp_reached_after_9_months
0,Apple Inc.,AAPL,2023-01-15,121.35,Hold,"Despite promising growth cost management, conc...",2023-01-16,2023-04-15,2023-07-15,2023-10-15,2024-01-15,133.504333,133.504333,164.873001,192.722244,195.186279,True,True,True
1,Apple Inc.,AAPL,2023-02-15,371.59,Buy,AAPL benefiting investment research developmen...,2023-02-16,2023-05-15,2023-08-15,2023-11-15,2024-02-15,154.117630,152.510254,172.393860,195.186279,188.734222,False,False,False
2,Apple Inc.,AAPL,2023-03-15,202.52,Hold,"AAPL's performance technology steady, risks su...",2023-03-16,2023-06-15,2023-09-15,2023-12-15,2024-03-15,151.795898,154.633575,184.813461,195.186279,197.361084,False,False,False
3,Apple Inc.,AAPL,2023-04-15,355.88,Hold,AAPL faces uncertainties shift consumer prefer...,2023-04-16,2023-07-15,2023-10-15,2024-01-15,2024-04-15,163.920502,163.920502,192.722244,195.186279,197.361084,False,False,False
4,Apple Inc.,AAPL,2023-05-15,292.45,Sell,AAPL's challenges expansion international mark...,2023-05-16,2023-08-15,2023-11-15,2024-02-15,2024-05-15,170.963104,170.963104,195.186279,188.734222,197.361084,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Exelon Corporation,EXC,2023-08-15,242.37,Sell,With ongoing issues like regulatory pressure e...,2023-08-16,2023-11-15,2024-02-15,2024-05-15,2024-08-15,38.032528,38.243870,40.194012,39.755924,38.337326,False,False,False
1232,Exelon Corporation,EXC,2023-09-15,248.48,Buy,EXC's continued growth environmental regulatio...,2023-09-16,2023-12-15,2024-03-15,2024-06-15,2024-09-15,40.011482,40.011482,40.011482,36.447979,38.337326,False,False,False
1233,Exelon Corporation,EXC,2023-10-15,148.36,Hold,EXC faces uncertainties shift consumer prefere...,2023-10-16,2024-01-15,2024-04-15,2024-07-15,2024-10-15,38.378361,38.589710,39.755924,37.114410,38.337326,False,False,False
1234,Exelon Corporation,EXC,2023-11-15,143.41,Buy,EXC benefiting labor shortages strong demand. ...,2023-11-16,2024-02-15,2024-05-15,2024-08-15,2024-11-15,38.466282,38.282047,39.755924,38.337326,38.376923,False,False,False


# Topic Modeling with LDA

# FINBERT

# KeyBERT

# LLM Argument Mining

- Provide List of pre-defined arguments